# Pore micorstructural and flow modelling of rocks.
### Estimating effective pore-throat radius from MICP data.

Lead Invsestigator: Olubukola Ishola (olubukola.ishola@okstate.edu)\
Co-Investigator: Javier Vilcaez\
Associated Paper: Augmenting Xray micro-CT data with MICP data for high resolution pore-microstructural and flow modelling of carbonate rocks.\
DOI:

---------------------------------------------------------------------------------------------------------------------------\
The notebook shows the principle behind the minimum incremental pore volume (MIPV) and estimating the effective pore-throat radius (EPTR) as outlined in steps in the paper above.\
Please see the paper for more details.

### Importing Packages

In [ ]:
import os
import numpy as np
import pandas as pd

### Step One

Account for the ink bottle effect. The shadowing or ink-bottle phenomenon causes overestimation of smaller pores since incremental pressure in MICP acquisition tends to mask large volume hidden behind tight pores as tighter pores. For this study, we applied a cut off at the minimum incremental pore volume (MIPV). MIPV represents the pore size where the incremental porosity abruptly reduces close to zero before attaining 100% mercury saturation. While this approach tries to remove false data, it is noteworthy that it also ignores pores sizes less than MIPV cut off.

In [ ]:
def mipv_cut_off(micp_data, threshold=0.99, show_data=True):
    """
    Apply the mipv cut-off to MICP data.

    Parameters
    ----------
    micp_data : float, dataframe
        Columns in the order : Pressure (psia), Pore Radius (µm), Incremental Pore Volume (mL/g), Cumulative Pore Volume (mL/g).
    threshold : float, Optional
        Threshold for applying mipv. Default is 0.99.
    show_data : bolean, Optional
        An option to show details of mipv cut-off point. Default is True.
        
    Returns
    -------
   micp_data : float, dataframe
       micp_data input truncated at the mipv cut-off. Columns in the order : Pressure (psia), Pore Radius (µm), Incremental Pore Volume (mL/g), Cumulative Pore Volume (mL/g). 

    """

    len_ = len(micp_data) - 2
    i = 0

    while i <= len_:
        curr = micp_data.iloc[i, 3] / micp_data.iloc[i + 1, 3]  # curr means current data point. The ratio of incremanetal pore volume at a given pressure to the next incremantal pore voulme

        if curr > threshold and curr < 1:
            after = micp_data.iloc[i + 1, 3] / micp_data.iloc[i + 2, 3]  # after means after current data point. The ratio of incremanetal pore volume at a given pressure to the next incremantal pore voulme

            if curr > after:

                if show_data:
                    print(micp_data.iloc[i + 1])

                return micp_data.iloc[:i + 2]

        i += 1  # increase counter for while loop

        if curr >= 1:
            i += 1  # If curr is less than one, mercury injected into the rock is increasing as expected. However, if it is greater than one, it indicates an error in data or mechanical issue during acquisition. To prevent further contamination of eptr estimation, skip the next iteration.

    return print(f'No cut-off found, lower threshold from {threshold}')

### Step Two

Estimate the weight ($w_i$) of each pore-throat size. In a previous work by Dastidar and others, the fraction of each incremental pore volume is used as weights.

$ w_i = \frac{v_i}{\sum_{i=1}^n v_i} \tag{1}$

Where $ n $ is the number of pore-throat size data point and  $ v_i $ is the incremental pore volume of a given pore-throat $ i $.

Here, we used the ratio of the incremental pore volume to the curved surface area ($ s $) of an equivalent cylinder as weights.

$ w_i = \frac{(v_i/s_i)}{\sum_{i=1}^n(v_i/s_i)} \tag{2}$

In [ ]:
def weights(incr_pore_vol, cur_surf_area_):
    """
    Estimate the weights of each pore-throat radius.

    Parameters
    ----------
    incr_pore_vol : float, 1D array
        A column of increamental pore volume.
    cur_surf_area_ : float, 1D array
        A column of surface area.
    
    Returns
    -------
    weights_ : float, 1D array
        Weights of the respective pore-throat size.

    """

    v_to_s = incr_pore_vol / cur_surf_area_
    sum_v_to_s = np.sum(v_to_s)
    weights_ = v_to_s / sum_v_to_s

    return weights_

curved surface area ($ s $) is given by

$ s_i=2πr_ih_i \tag{3}$

Where $r_i$ is the radius of a given pore-throat $i$ and $h_i$ is corresponding the pore throat length. In this study we assumed $h_i$ to be constant, hence, $s_i$ is controlled by $r_i$ in Eq. (2).



In [ ]:
def cur_surf_area(pore_throat_radius, pore_throat_length=1):
    """
    Estimate the surface area of each pore-throat radius.

    Parameters
    ----------
    pore_throat_radius : float, 1D array
        A column of pore-throat radius.
    pore_throat_length : float, 1D array
        A column of pore_throat length.
    
    Returns
    -------
    cur_surf_area_ : float, 1D array
        A column of surface area.

    """

    cur_surf_area_ = 2 * 3.14 * pore_throat_radius * pore_throat_length

    return cur_surf_area_

The incremental pore volume ($v_i$) accounts for the fraction of total volume of fluid a pore throat ($i$) controls resulting in that pore throat size having more influence on flow through the rock sample. The curved surface area $s_i$ is introduced in this study to account for the impact of frictional interaction at the fluid-rock interface which negates flow through a pore-throat $i$. Curved surface area and not total surface area is used here because the pore throats are idealized as cylinders and the portion of the cylinder having the frictional interaction with the fluid flowing through is the curved surface since the idealized cylinder is expected to be hollow for fluid to pass through it. 

### Step Three

Calculate the $EPTR$ by using the estimated weights to compute a weighted average pore throat radius (Eq. 4).

$EPTR = \frac{\sum_{i=1}^n(w_i×r_i)}{\sum_{i=1}^n(w_i)} \tag{4}$

In [ ]:
def eptr(weights_, pore_throat_radius):
    """
    Estimate the eptr.

    Parameters
    ----------
    weights_ : float, 1D array
        Weights of the respective pore-throat size.
    pore_throat_radius : float, 1D array
        A column of pore-throat radius.
    
    Returns
    -------
    eptr : float
        Equivalent pore-throat.

    """
    eptr_ = np.sum(
        weights_ * pore_throat_radius
    )  # denominator is ignored becuase it mathematically equals 1.

    return eptr_

### Inputs

In [ ]:
filename = 'micp_a.xlsx'
file_path = r'../micp_data'

### Load data

In [ ]:
file = os.path.join(file_path, filename)
df = pd.read_excel(file)
df = mipv_cut_off(df)
pore_throat_radius = df.iloc[:, 1]
incr_pore_vol = df.iloc[:, 2]

### Run Program

In [ ]:
cur_surf_area_ = cur_surf_area(pore_throat_radius, pore_throat_length=1)
weights_ = weights(incr_pore_vol, cur_surf_area_)
eptr_ = eptr(weights_, pore_throat_radius)
print(
    f"The equivalent pore_throat radius of the sample is {round(eptr_, 3)} microns."
)